## Pix2pix image colorization. 
#### Examples using pre-trained generator model and a collection of images that are in the training and outside the training set. Also getting results analyzed via histogram comparsions and attempting to colorize webcam mjpeg stream in real time.

https://github.com/skirdey/mxnet-pix2pix - code to train pix2pix using MXNet and Python 3.6, it also visualizes training process via cv2 library

The network was trained on a set of 36000 256x256 images mostly covering city streets and city scape. The images were extracted from a video at about 4fps. The training process took approximately 20 hours on a laptop with Nvidia Geforce GTX 1060 with 6GB of video memory. 

Skinny of the tutorial:
1. Load an image
2. Convert to lab and extract lightness
3. Run lightness through pre-trained pix2pix generator
4. Display and analyize colorization result
5. Colorize video captchured from a live web-cam in real-time and display an example. 

#### Demo of real-time colorization ( not real time anymore ). Video stream came from a web camera attached to the computer. 
https://s3-us-west-1.amazonaws.com/pix2pix/real_time_video_colorization_street.gif

In [1]:
from IPython.display import HTML
HTML('<img src="https://s3-us-west-1.amazonaws.com/pix2pix/real_time_video_colorization_street.gif">')

#### Demo of real-time colorization ( not real time anymore ). Video stream came from an online web camera.
https://s3-us-west-1.amazonaws.com/pix2pix/real_time_video_colorization.gif

In [2]:
from IPython.display import HTML
HTML('<img src="https://s3-us-west-1.amazonaws.com/pix2pix/real_time_video_colorization.gif">')

#### Let's start!

In [3]:
%matplotlib inline
import os
import glob
import cv2
import time
import mxnet as mx
import mxnet.ndarray as nd
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

from cv2_utils import show_mxnet_to_numpy_array, show_numpy_array
from util import rgb_to_lab
from util.process_lab_utils_mx import preprocess_lab
from util.process_lab_utils_np import lab_parts_to_rgb

Importing necessary utilities and system modules. We are doing lab colorization, meaning we will convert an image into the lab space and use the lightness channel to run it through a generator. The network, traning, and helper classed can be found here - https://github.com/skirdey/mxnet-pix2pix

In [7]:
ctx = mx.cpu(0) # Nvidia Geforce GTX 1060 6GB. You should see games run on this beauty. 

#### Helper functions
What we are doing here is loading a pre-trained generator model, and binding data parameters to it.
We also have several functions to load an image, either jpeg or png, and get is lightness channel that we can run through generator network. 

In [8]:
def timeme(method):
    def wrapper(*args, **kw):
        startTime = time.time()
        result = method(*args, **kw)
        endTime = time.time()
        print(str(method.__name__) + ": ", endTime - startTime , 'seconds')
        return result
    return wrapper

In [9]:
def load_generator_from_checkpoint():
    mod_generator = mx.module.Module.load("G", 7, load_optimizer_states=True, data_names=('gData',),
                                               label_names=None, context=ctx)
    mod_generator.bind(data_shapes=[('gData', (1,) + (1, 256, 256))], for_training=False)
    return mod_generator

In [10]:
def load_image(image_filename):
    assert image_filename
    with open(image_filename, 'rb') as fp:
        str_image = fp.read()

    return mx.img.imdecode(str_image)

##### Note
Get color channels function uses internal helper functions to process an RGB image into LAB space. 
It also resizes an image into 256 by 256 square so we can run it through the generator network. 
Before returning the result, we resize the generator's output back to image's original size. 
As generator expects a square image, it is best to operate on square sample images to test out colorization. 

In [11]:
@timeme
def generator_run(model, grayscale_lightness):
    model.forward(mx.io.DataBatch([grayscale_lightness]), is_train=True)
    return model.get_outputs()[0].copy()

In [12]:
def get_color_channels(image, model):

    w, h = image.shape[0:2]

    if image.shape[0:2] != (256, 256):
        image = mx.image.resize_short(image, 256)
        image = mx.image.center_crop(image, (256, 256))[0]


    lightness_chan = get_lightness(image)

    grayscale_lightness = nd.expand_dims(lightness_chan, axis=2).transpose((2, 3, 0, 1))

    output = generator_run(model, grayscale_lightness)
    
    a_b_channels = squeeze_mx_image(output).transpose((2,1,0))

    a_b_channels = cv2.resize(a_b_channels.asnumpy(), (w, h))
    a_b_channels = nd.expand_dims(nd.array(a_b_channels), axis=2).transpose((2, 3, 1, 0))

    return a_b_channels

In [13]:
def squeeze_mx_image(image):
    return nd.array(np.squeeze(image.asnumpy(), axis=0), ctx=ctx)

In [14]:
def get_lightness(image):
    image = image.as_in_context(ctx)
    lab = rgb_to_lab(image, ctx=ctx)
    lightness_chan, _, _ = preprocess_lab(lab)
    return lightness_chan

In [15]:
def colorize_image(real_a, model):
    color_channels = get_color_channels(real_a, model)
    original_size_lightness = get_lightness(real_a)

    grayscale_lightness = nd.expand_dims(original_size_lightness, axis=2).transpose((3,2,0,1))

    fake_rgb = lab_parts_to_rgb(color_channels.asnumpy() * 3, grayscale_lightness.asnumpy())
    
    return fake_rgb

#### Moment of truth
Let's load an image that I've taken on my phone, cropped it to a square and resized to 512x512. The neural net never seen this picture before. It was quite new for me as well - sunny in Seattle!

#### A bit on Pix2pix
Pix2pix approach is based on two neural networks, that are trained at the same time. 

Generator network in a original GAN is a model that learns mapping from random noise vector $z$ to output image $y$:

$$G : z \rightarrow y$$

Pix2pix uses approach of conditional GANS, where generator learns mapping from original image $x$ and random noise vector $z$ to original image $y$:

$$ G : {x, z} \rightarrow y$$

The objective of a pix2pix training is $G^{∗}= arg\,\underset{G}{min}\,\underset{D}{max}\,\mathcal{L}_{cGAN} (G, D) \,+\, \alpha\times \mathcal{L}_{L1}(G)$

The objective stands for use gradient 'ascent' on Discriminator and *increase* the probability of detecting fakes and recognizes real pictures, and use gradient descent on the generator to decrease the probability to be detected as a fake. 

In [ ]:
model = load_generator_from_checkpoint()

real = load_image("./test_img/IMG-2269.jpg")
colorized =  colorize_image(nd.cast(real, "uint8"), model)

In [ ]:
# Original, straight from the phone
real = real.asnumpy() # It has type MXNet.NDArray at this point
Image.fromarray(real)

In [ ]:
# Colorized version
Image.fromarray(colorized)

#### Metrics
It is nice to see the result visually. And it is nice to see some quanitifed metrics. We can have rgb histogram comparsions between original and colorized version, and also an L1 difference between the fake and the real images. It also make sense to time the colorization call itself, and see how fast it performs its task. Lets start with it:

#### Histograms

In [ ]:

color = ('r','g','b')
plt.figure(figsize=(15,4))
for channel, col in enumerate(color):
    histr_real = cv2.calcHist([real], [channel], None, [256], [0,256])
    histr_colorized = cv2.calcHist([colorized], [channel], None, [256], [0,256])
    
    plt.xlim([0,256])
    plt.ylim([-500, 500])
    plt.subplot(1, 3, 1) # plt.subplot allows us to plot side by side, the args are (row, col, current plot)
    plt.title("Real")
    plt.plot(histr_real,color = col)
    
    plt.subplot(1,3,2)
    plt.title("Colorized")
    plt.plot(histr_colorized, color = col)
    
    plt.subplot(1,3,3)
    plt.title("Difference")
    plt.plot(histr_real - histr_colorized, color = col)
    

plt.show()

Just for the test, lets make sure the histogram of real image has correlation of 1 
to itself, and do the same for the colorized version 

Let's compute the difference numerically using OpenCV compareHist function. 
https://docs.opencv.org/2.4/modules/imgproc/doc/histograms.html?highlight=comparehist#comparehist

In [ ]:
# Comparing all three color channels at once
histr_real = cv2.calcHist([real], [0,1,2], None, [256, 256, 256], [0,256, 0, 256, 0, 256])
histr_colorized = cv2.calcHist([colorized], [0,1,2], None, [256, 256, 256], [0,256, 0, 256, 0, 256])


cv2.compareHist(histr_real, histr_real, 0)

In [ ]:
cv2.compareHist(histr_colorized, histr_colorized, 0)

At least we know that correlation functions works on two identical histograms. 
Lets try it on real and colorized images. Correlatiom method performs following computation:

### Correlation metric
[-1:1], 1 being perfect match

$$d(H_1,H_2) =  \frac{\sum_I (H_1(I) - \bar{H_1}) (H_2(I) - \bar{H_2})}{\sqrt{\sum_I(H_1(I) - \bar{H_1})^2 \sum_I(H_2(I) - \bar{H_2})^2}}$$ where $$\bar{H_k} =  \frac{1}{N} \sum _J H_k(J)$$ and $N$ is a total number of bins. 

In [ ]:
cv2.compareHist(histr_real, histr_colorized, 0)

-0.27 - Not great, but not bad. The colorized image looks like a badly taken photo, but the colors there do make sense.

### Chi-Square
[0: positive infinity], 0 is perfect match
It computes $$d(H_1,H_2) =  \sum _I  \frac{\left(H_1(I)-H_2(I)\right)^2}{H_1(I)}$$

In [ ]:
cv2.compareHist(histr_real, histr_colorized, 1)

### Bhattacharyya distance
[-1: 1], 0 is perect match
$$d(H_1,H_2) =  \sqrt{1 - \frac{1}{\sqrt{\bar{H_1} \bar{H_2} N^2}} \sum_I \sqrt{H_1(I) \cdot H_2(I)}}$$

In [ ]:
cv2.compareHist(histr_real, histr_colorized, 3)

0.8798752374385485 - far away from a perfect match. Still, visually colorization result doesn't feel that wrong. 

### Video colorization on the client in a browser

In [ ]:
%matplotlib notebook
import signal

In [ ]:
def signal_handler(signal, frame):
    # KeyboardInterrupt detected, exiting
    global is_interrupted
    is_interrupted = True

In [ ]:
one_more_url = "http://217.197.157.7:7070/axis-cgi/mjpg/video.cgi?resolution=320x240"
url = "http://85.46.64.147/axis-cgi/mjpg/video.cgi?camera=&resolution=320x240" # looks like italy
vc = cv2.VideoCapture(url)

plt.ion()

if vc.isOpened(): # try to get the first frame
    is_capturing, frame = vc.read()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    frame_colorized = colorize_image(nd.array(frame), model)
    
    plt.figure(figsize=(15,5))
    plt.subplot(1, 2, 1)
    webcam_preview_original = plt.imshow(frame)
    plt.subplot(1, 2, 2)
    webcam_preview_colorized = plt.imshow(frame_colorized)    
else:
    is_capturing = False

signal.signal(signal.SIGINT, signal_handler)
is_interrupted = False
while is_capturing:
    is_capturing, frame = vc.read() 
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    frame_colorized = colorize_image(nd.array(frame), model)
    
    webcam_preview_original.set_data(frame)
    webcam_preview_colorized.set_data(frame_colorized)
    
    plt.show()

    try:    # Avoids a NotImplementedError caused by `plt.pause`
        plt.pause(0.05)
    except Exception:
        pass
    if is_interrupted:
        vc.release()
        break

##### Let's take a look at a recorded gif once again

In [ ]:
from IPython.display import HTML
HTML('<img src="https://s3-us-west-1.amazonaws.com/pix2pix/real_time_video_colorization.gif">')

#### I hope you enjoyed the tutorial. It will probably be updated with better metrics and more video examples. 